In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from  feature_scaler import *

# Model 1

In this notebook Model 1 is tested.

## Import data & prepare dataset: 

In [2]:
#Declare which dataset want to evaluate; hyperparameter for evaluation
dataset_nr = 5

X_train = pd.read_csv(f"Inputs/Dataset_{dataset_nr}/X_train.csv")
y_train = pd.read_csv(f"Inputs/Dataset_{dataset_nr}/y_train.csv")
X_val = pd.read_csv(f"Inputs/Dataset_{dataset_nr}/X_val.csv")
y_val = pd.read_csv(f"Inputs/Dataset_{dataset_nr}/y_val.csv")
X_test = pd.read_csv(f"Inputs/Dataset_{dataset_nr}/X_test.csv")
y_test = pd.read_csv(f"Inputs/Dataset_{dataset_nr}/y_test.csv")

In [ ]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Input
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import mean_absolute_error
import time


# Define the model creation function
def create_model(n_features):
    model = Sequential() # Define a sequential model (FNN)
    model.add(Input(shape=(n_features,))) # Input layer
    model.add(Dense(128, activation="relu"))  # layer 1
    model.add(Dense(64, activation="relu"))  # layer 2
    model.add(Dense(1, activation="linear"))  # output layer
    model.compile(optimizer='adam', loss="mean_squared_error", metrics=["mean_absolute_error"])
    return model


#Define input shape
n_features = X_train.shape[1]

#Initialize results list
results = [] 

# Train and evaluate the model 10 times
for run in range(1, 11):
    print(f"Run {run}")
    start_time = time.time() #Track running time
    
    model = create_model(n_features) 
    
    #Define checkpoint: Saves model at epoch with best validation loss
    checkpoint = ModelCheckpoint(f'learned_models/Model_1/data_{dataset_nr}_run_{run}.keras', monitor='val_loss', save_best_only=True, mode='min') 
    # Early stopping after 5 epochs without improvement
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    model.fit(X_train.values, y_train.values, validation_data=(X_val.values, y_val.values), epochs=20, batch_size=256, callbacks=[checkpoint], verbose = 0)
    
    #Save best model for prediction (important beacuse overfitting may cause the best model to be in the middle of the training process)
    best_model = load_model(f'learned_models/Model_1/data_{dataset_nr}_run_{run}.keras')   #update with each dataset!
    
    # Evaluate the best model of each run at the end
    y_pred = best_model.predict(X_test.values) # Predictions of test data
    mae = mean_absolute_error(y_test.values, y_pred) # MAE of test data
    
    end_time = time.time() 
    run_time = end_time - start_time #Calculate run time of each run
    
    results.append((run, mae, run_time))

# Store results in a DataFrame
results_df = pd.DataFrame(results, columns=['Run', 'MAE', 'Run Time'])

# Save the dataframe
results_df.to_csv(f'Results_df/Model_1/Data_{dataset_nr}.csv', index=False)


# Compute average MAE, standard deviation, and average running time
average_mae = results_df['MAE'].mean()
std_mae = results_df['MAE'].std()
average_run_time = results_df['Run Time'].mean()

print("Results DataFrame:")
print(results_df)
print(f"Average MAE: {round(average_mae, ndigits=2)}")
print(f"Standard Deviation of MAE: {round(std_mae, ndigits=2)}")
print(f"Average Running Time: {round(average_run_time, ndigits=2)} seconds")

In [ ]:
#Print architecture of model
model.summary()

## Testing performance on new customers

In [ ]:
from keras.losses import mean_absolute_error
from keras.models import load_model
from sklearn.metrics import mean_absolute_error


dataset_nr = 5
X_test = pd.read_csv(f"Inputs/Dataset_{dataset_nr}/X_test_all_customers.csv")
y_test = pd.read_csv(f"Inputs/Dataset_{dataset_nr}/y_test_all_customers.csv")
X_test_subset = pd.read_csv(f"Inputs/Dataset_{dataset_nr}/X_test.csv")
y_test_subset = pd.read_csv(f"Inputs/Dataset_{dataset_nr}/y_test.csv")


#Define model learned from cell above
best_model = load_model('learned_models/Model_1/data_5_run_10.keras')

#Predictions on X_test subset
predicted_values_subset = best_model.predict(X_test_subset.values)

# Make predictions on X_test
predicted_values = best_model.predict(X_test.values)

# Convert the predicted values to a DataFrame
predicted_df_subset = pd.DataFrame(predicted_values_subset, columns=['Predicted'])
actual_df_subset = pd.DataFrame(y_test_subset.values, columns=['Actual'])
df_comparison_subset = pd.concat([actual_df_subset, predicted_df_subset], axis=1)




# Convert the predicted values to a DataFrame
predicted_df = pd.DataFrame(predicted_values, columns=['Predicted'])
# Optionally, add the actual values for comparison
actual_df = pd.DataFrame(y_test.values, columns=['Actual'])
df_comparison = pd.concat([actual_df, predicted_df], axis=1)


print("Shape of X_test without new customers: ", X_test_subset.shape)
print("Shape of X_test with new customers: ", X_test.shape)
print('------------------------------------------------')
MAE_subset = mean_absolute_error(df_comparison_subset['Actual'], df_comparison_subset['Predicted'])
print(f"Mean Absolute Error without new customers: {round(MAE_subset, ndigits= 2)}")
print('------------------------------------------------')
MAE = mean_absolute_error(df_comparison['Actual'], df_comparison['Predicted'])
print(f"Mean Absolute Error with new customers: {round(MAE, ndigits= 2)}")
print('------------------------------------------------')